In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/dataset_1.csv")
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
label2id = {
    "DDoS": 0,
    "PortScan": 1,
    "BENIGN": 2,
}
df["Label"] = df["Label"].map(label2id)
Y = df["Label"].values
X = df.drop(columns=["Label", "IP"]).values
X_train1, *_ = train_test_split(X, Y, test_size=0.2, random_state=42)

df = pd.read_csv("data/dataset_2.csv")
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
label2id = {
    "DDoS": 0,
    "PortScan": 1,
    "BENIGN": 2,
}
df["Label"] = df["Label"].map(label2id)
Y = df["Label"].values
X = df.drop(columns=["Label", "IP"]).values
X_train2, *_ = train_test_split(X, Y, test_size=0.2, random_state=42)

X = np.concatenate((X_train1, X_train2), axis=0)
scaler = StandardScaler()
scaler.fit(X)
# Save scaler
np.save("data/scaler.npy", scaler)
# Load scaler
scaler = np.load("data/scaler.npy", allow_pickle=True).item()
X = scaler.transform(X)

In [2]:
import numpy as np
from phe import paillier

# Generate keypair
public_key, private_key = paillier.generate_paillier_keypair()

# Save keypair
np.save("data/public_key.npy", public_key)
np.save("data/private_key.npy", private_key)

# Load keypair
public_key = np.load("data/public_key.npy", allow_pickle=True).item()
private_key = np.load("data/private_key.npy", allow_pickle=True).item()


a = np.array([1, 2, 3, 4, 5], dtype=np.float64)
b = np.array([2, 3, 4, 5, 6], dtype=np.float64)

enc_operator = np.frompyfunc(public_key.encrypt, 1, 1)
dec_operator = np.frompyfunc(private_key.decrypt, 1, 1)

enc_a = enc_operator(a)
enc_b = enc_operator(b)

enc_sum = enc_a + enc_b
res = dec_operator(enc_sum)
print(res)

[3.0 5.0 7.0 9.0 11.0]
